# Scheduling Problem by Sregneva
In this code we will be implementing CSP (Constraint Satifaction Problem) to solve a scheduling problem using the UCS (Un-Constrained Scheduling) type of algorithm, where we will take the first solution that is found.

We are using Ortools. A library created by Google to help solve the model that we will create. (https://developers.google.com/optimization/install/python)
Constraints that are used:
1. One doctor and one nurse per shift
2. No doctor or nurse works more than one shift in one day
3. No doctor or nurse who worked the latest shift works the next day earliest shift
4. No specialist works on the latest shift

## Setting Up The Model

### Importing the Library

First, we need to import cp_model from ortools.sat.python.

In [1]:
#!pip install ortools
from ortools.sat.python import cp_model

### Variables Initialization

Here we will initialize the variables that we need.
We ask for user, which is thought as admin here, to input the number of nurses, the number of general doctors, the number of specialists, the number of days, and the number of shift wanted.
We sum up the general doctor and specialist to make only one schedule for the doctors, when we need to check if the doctor is a specialist or not, we will check them by their index.
After that we will create a range variable for each of those inputed variables.<br>
We also need to initialize a model from the cp_model that we have imported

The input for must be following these rules:
1. There has to be more than 1 shift
2. There has to be more than 1 nurse
3. There has to be more than 1 general doctor/specialist
4. There has to be more than or equal to the number of nurses and the number of doctors

In [2]:
def input_checker(inputed):
    if inputed <= 1:
        inputed = input_checker(int(input("The number you entered is violating the input rules." +
                                          "Please re-enter the number.\n")))
    return inputed

def input_checker_shift(inputed):
    if inputed <= 1 or inputed > 4:
        inputed = input_checker(int(input("The number you entered is violating the input rules." +
                                          "Please re-enter the number.\n")))
    return inputed

def input_checker_nurse(num_nurses, num_shifts):
    if num_nurses <= 1 or num_nurses < num_shifts:
        num_nurses = input_checker_nurse(int(input("The number you entered is violating the input rules." +
                                          "Please re-enter the number.\n")), num_shifts)
    return num_nurses

def input_checker_doc(num_doctor_general, num_doctor_special, num_shifts):
    if num_doctor_general + num_doctor_special <= 1 or num_doctor_general + num_doctor_special < num_shifts:
        num_doctor_general = int(input("The number you entered is violating the input rules. " +
                                       "Please re-nter the number.\n\nEnter number of genral doctors:\n"))
        num_doctor_special = int(input("Enter number of specialist doctors:\n"))
        input_checker_doc(num_doctor_general, num_doctor_special, num_shifts)
    return num_doctor_general, num_doctor_special

In [3]:
#Variable initialization
num_shifts = input_checker_shift(int(input("Enter number of shifts:\n")))
num_days = input_checker(int(input("Enter number of days:\n")))

num_doctor_general = int(input("Enter number of general doctors:\n"))
num_doctor_special = int(input("Enter number of specialist doctors:\n"))
num_doctor_general, num_doctor_special = input_checker_doc(num_doctor_general, num_doctor_special, num_shifts)
num_doctors = (num_doctor_general + num_doctor_special)

num_nurses = input_checker_nurse(int(input("Enter number of nurses:\n")), num_shifts)

all_nurses = range(num_nurses)
all_doctors = range(num_doctors)
all_shifts = range(num_shifts)
all_days = range(num_days)

# Creates the model.
model = cp_model.CpModel()

Enter number of shifts:
3
Enter number of days:
7
Enter number of general doctors:
3
Enter number of specialist doctors:
2
Enter number of nurses:
5


Next, we need to initialize a set for the nurses and the doctors.

In [4]:
# Creates shift variables.
# nurse_shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
nurse_shifts = {}
for n in all_nurses:
    for d in all_days:
        for s in all_shifts:
            nurse_shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))
            
# doctor_shifts[(n, d, s)]: doctor 'n' works shift 's' on day 'd'.
doctor_shifts = {}
for n in all_doctors:
    for d in all_days:
        for s in all_shifts:
            doctor_shifts[(n, d, s)] = model.NewBoolVar('shift_dc%id%is%i' % (n, d, s))

## Adding Constraints to The Model

As mentioned above, we use the following constraints:
1. One doctor and one nurse per shift
2. No doctor or nurse works more than one shift in one day
3. No doctor or nurse who worked the latest shift works the next day earliest shift
4. No specialist works on the latest shift

All while trying to make a schedule that is as even as possible to every nurse and every doctor.

In [5]:
# Adding the constraints
# Each shift is assigned to exactly one nurse and one doctor.
for d in all_days:
    for s in all_shifts:
        model.Add(sum(nurse_shifts[(n, d, s)] for n in all_nurses) == 1)
        model.Add(sum(doctor_shifts[(n, d, s)] for n in all_doctors) == 1)

for n in all_nurses:
    for d in all_days:
        # Each nurse works at most one shift per day.
        model.Add(sum(nurse_shifts[(n, d, s)] for s in all_shifts) <= 1)
        
        #Each nurse who works on the last shift 
        #will not work on the first shift of the following day
        model.Add(sum([nurse_shifts[(n, d, num_shifts - 1)], nurse_shifts[(n, (d + 1) % num_days, 0)]]) <= 1)
        
for n in all_doctors:
    for d in all_days:
        # Each doctor works at most one shift per day.
        model.Add(sum(doctor_shifts[(n, d, s)] for s in all_shifts) <= 1)
        
        #Each doctor who works on the last shift 
        #will not work on the first shift of the following day
        model.Add(sum([doctor_shifts[(n, d, num_shifts - 1)], doctor_shifts[(n, (d + 1) % num_days, 0)]]) <= 1)

#No specialist doctor will work on the last shift
for n in range(num_doctors - num_doctor_special, num_doctors):
    for d in all_days:
        model.Add(sum([doctor_shifts[(n, d, num_shifts - 1)]]) == 0)
        
# Try to distribute the shifts evenly, so that each nurse and each doctor works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses or by the number of doctors,
# some nurses and some doctors will be assigned one more shift.
min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
if num_shifts * num_days % num_nurses == 0:
    max_shifts_per_nurse = min_shifts_per_nurse
else:
    max_shifts_per_nurse = min_shifts_per_nurse + 1
for n in all_nurses:
    num_shifts_worked = 0
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked += nurse_shifts[(n, d, s)]
    model.Add(min_shifts_per_nurse <= num_shifts_worked)
    model.Add(num_shifts_worked <= max_shifts_per_nurse)
    
min_shifts_per_doctor = (num_shifts * num_days) // num_doctors
if num_shifts * num_days % num_doctors == 0:
    max_shifts_per_doctor = min_shifts_per_doctor
else:
    max_shifts_per_doctor = min_shifts_per_doctor + 1
for n in all_doctors:
    num_shifts_worked = 0
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked += doctor_shifts[(n, d, s)]
    model.Add(min_shifts_per_doctor <= num_shifts_worked)
    model.Add(num_shifts_worked <= max_shifts_per_doctor)

## Solving and Printing

### Print All

Here, we will print all schedule.

In [6]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print solutions."""

    def __init__(self, nurse_shifts, num_nurses, doctor_shifts, num_doctors, num_days, num_shifts, num_doctor_general):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._nurse_shifts = nurse_shifts
        self._num_nurses = num_nurses
        self._doctor_shifts = doctor_shifts
        self._num_doctors = num_doctors
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._num_doctor_general = num_doctor_general
        
    def day_converter(day):
        if day%7 == 0:
            return "Monday"
        elif day%7 == 1:
            return "Tuesday"
        elif day%7 == 2:
            return "Wednesday"
        elif day%7 == 3:
            return "Thursday"
        elif day%7 == 4:
            return "Friday"
        elif day%7 == 5:
            return "Saturday"
        elif day%7 == 6:
            return "Sunday"
        
    def on_solution_callback(self):
        string_output = ""
        for d in range(self._num_days):
            string_output +=(SolutionPrinter.day_converter(d) + "\n")
            string_temp2 = ""
            for s in range(self._num_shifts):
                string_temp2 += ("\tShift " + str(s+1) + "\n")
                is_nurse_working = False
                is_doctor_working = False
                string_temp3 = ""
                for n in range(self._num_nurses):
                    if self.Value(self._nurse_shifts[(n, d, s)]):
                        is_nurse_working = True
                        string_temp3 += ("\t\tNurse " + str(n+1))
                for n in range(self._num_doctors):
                    if self.Value(self._doctor_shifts[(n, d, s)]):
                        is_doctor_working = True
                        if int(n) > self._num_doctor_general-1:
                            string_temp3 += (" Doctor " + str(n+1) + " (Specialist)\n")
                        else:
                            string_temp3 += (" Doctor " + str(n+1) + "\n")
                string_temp2 += string_temp3
            string_output += string_temp2
        self.StopSearch()
        print(string_output)

In [7]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
solution_printer = SolutionPrinter(nurse_shifts, num_nurses, doctor_shifts, 
                                   num_doctors, num_days, num_shifts, num_doctor_general)
solver.SolveWithSolutionCallback(model, solution_printer)

Monday
	Shift 1
		Nurse 5 Doctor 4 (Specialist)
	Shift 2
		Nurse 3 Doctor 5 (Specialist)
	Shift 3
		Nurse 4 Doctor 3
Tuesday
	Shift 1
		Nurse 5 Doctor 4 (Specialist)
	Shift 2
		Nurse 4 Doctor 5 (Specialist)
	Shift 3
		Nurse 1 Doctor 3
Wednesday
	Shift 1
		Nurse 5 Doctor 4 (Specialist)
	Shift 2
		Nurse 4 Doctor 5 (Specialist)
	Shift 3
		Nurse 3 Doctor 1
Thursday
	Shift 1
		Nurse 5 Doctor 5 (Specialist)
	Shift 2
		Nurse 2 Doctor 4 (Specialist)
	Shift 3
		Nurse 1 Doctor 2
Friday
	Shift 1
		Nurse 3 Doctor 1
	Shift 2
		Nurse 1 Doctor 3
	Shift 3
		Nurse 2 Doctor 2
Saturday
	Shift 1
		Nurse 3 Doctor 3
	Shift 2
		Nurse 1 Doctor 1
	Shift 3
		Nurse 2 Doctor 2
Sunday
	Shift 1
		Nurse 1 Doctor 3
	Shift 2
		Nurse 4 Doctor 2
	Shift 3
		Nurse 2 Doctor 1



4

### Print Requested Day

Here, we can request a spesific day to print.

In [8]:
string_output_global = ""
class SolutionPrinterRequestedDay(cp_model.CpSolverSolutionCallback):
    """Print solutions."""

    def __init__(self, nurse_shifts, num_nurses, doctor_shifts, num_doctors, requested_day, num_shifts,
                 num_doctor_general, num_doctor_special):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._nurse_shifts = nurse_shifts
        self._num_nurses = num_nurses
        self._doctor_shifts = doctor_shifts
        self._num_doctors = num_doctors
        self._day = requested_day
        self._num_shifts = num_shifts
        self._num_doctor_general = num_doctor_general
        self._num_doctor_special = num_doctor_special

    def day_converter(day):
        if day%7 == 0:
            return "Monday"
        elif day%7 == 1:
            return "Tuesday"
        elif day%7 == 2:
            return "Wednesday"
        elif day%7 == 3:
            return "Thursday"
        elif day%7 == 4:
            return "Friday"
        elif day%7 == 5:
            return "Saturday"
        elif day%7 == 6:
            return "Sunday"
        
    def on_solution_callback(self):
        d = self._day
        string_output = (SolutionPrinterRequestedDay.day_converter(d) + "\n")
        string_temp2 = ""
        for s in range(self._num_shifts):
            string_temp2 += ("\tShift " + str(s+1) + "\n")
            is_nurse_working = False
            is_doctor_working = False
            string_temp3 = ""
            for n in range(self._num_nurses):
                if self.Value(self._nurse_shifts[(n, d, s)]):
                    is_nurse_working = True
                    string_temp3 += ("\t\tNurse " + str(n+1))
            for n in range(self._num_doctors):
                if self.Value(self._doctor_shifts[(n, d, s)]):
                    is_doctor_working = True
                    if int(n) > self._num_doctor_general - 1:
                        string_temp3 += (" Doctor " + str(n+1) + " (Specialist)\n")
                    else:
                        string_temp3 += (" Doctor " + str(n+1) + "\n")
            string_temp2 += string_temp3
        string_output += string_temp2
        global string_output_global
        string_output_global = string_output
        self.StopSearch()
        print(string_output)

In [9]:
req_day = int(input("Enter day: [in number ex: 0; 2]\n"))

Enter day: [in number ex: 0; 2]
4


In [10]:
solution_printer = SolutionPrinterRequestedDay(nurse_shifts, num_nurses, doctor_shifts, num_doctors,
                                               req_day, num_shifts,  num_doctor_general, num_doctor_special)
solver.SearchForAllSolutions(model, solution_printer)

Friday
	Shift 1
		Nurse 3 Doctor 1
	Shift 2
		Nurse 1 Doctor 3
	Shift 3
		Nurse 2 Doctor 2



2

### Print Requested Day and Doctor

Here, we can request a spesific day and type of doctor.
When the chosen type of doctor is not on call, the patient will get an error message.

In [11]:
# string_output_global = ""
class SolutionPrinterRequestedDayAndDoc(cp_model.CpSolverSolutionCallback):
    """Print solutions."""

    def __init__(self, nurse_shifts, num_nurses, doctor_shifts, num_doctors, requested_day, num_shifts, 
                 num_doctor_general, num_doctor_special, doctor_type):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._nurse_shifts = nurse_shifts
        self._num_nurses = num_nurses
        self._doctor_shifts = doctor_shifts
        self._num_doctors = num_doctors
        self._day = requested_day
        self._num_shifts = num_shifts
        self._num_doctor_general = num_doctor_general
        self._num_doctor_special = num_doctor_special
        self._doctor_type = doctor_type

    def day_converter(day):
        if day%7 == 0:
            return "Monday"
        elif day%7 == 1:
            return "Tuesday"
        elif day%7 == 2:
            return "Wednesday"
        elif day%7 == 3:
            return "Thursday"
        elif day%7 == 4:
            return "Friday"
        elif day%7 == 5:
            return "Saturday"
        elif day%7 == 6:
            return "Sunday"    
    
    def on_solution_callback(self):
        d = self._day
        string_output = (SolutionPrinterRequestedDayAndDoc.day_converter(d) + "\n")
        string_temp2 = ""
        req_checker = False
        for s in range(self._num_shifts):
            string_temp2 += ("\tShift " + str(s+1) + "\n")
            is_nurse_working = False
            is_doctor_working = False
            string_temp3 = ""
            for n in range(self._num_nurses):
                if self.Value(self._nurse_shifts[(n, d, s)]):
                    is_nurse_working = True
                    string_temp3 += ("\t\tNurse " + str(n))
            for n in range(self._num_doctors):
                if self.Value(self._doctor_shifts[(n, d, s)]):
                    is_doctor_working = True
                    if int(n) > self._num_doctor_general - 1:
                        string_temp3 += (" Doctor " + str(n) + " (Specialist)\n")
                        if(self._doctor_type == "Spesialis"):
                            req_checker = True
                    else:
                        string_temp3 += (" Doctor " + str(n) + "\n")
                        if(self._doctor_type == "Umum"):
                            req_checker = True
            string_temp2 += string_temp3
        string_output += string_temp2
        self.StopSearch()
        if (req_checker == True):
            string_akhir = "We found a match!\n" + string_output
            string_output_global = string_akhir
            print(string_akhir)
        else:
            #global string_output_global
            string_output_global = "Sorry, no schedule match your request, please try for another day."
            print(string_output_global)

In [12]:
req_day = int(input("Enter wanted day: [in number ex: 0; 2]\n"))
req_doc = input("Enter wanted doctor:\n[Umum for general doctor; Spesialis for specialist]\n")
symptom = input("Enter symptoms:\n")

Enter wanted day: [in number ex: 0; 2]
5
Enter wanted doctor:
[Umum for general doctor; Spesialis for specialist]
Spesialis
Enter symptoms:
Fever


In [13]:
solution_printer = SolutionPrinterRequestedDayAndDoc(nurse_shifts, num_nurses, doctor_shifts, num_doctors, req_day, num_shifts,
                                                     num_doctor_general, num_doctor_special, req_doc)
solver.SearchForAllSolutions(model, solution_printer)

Sorry, no schedule match your request, please try for another day.


2

## Constraints checker

Here, we can double check if our constraints are working

In [14]:
def ConstraintChecker():
    list_result = []
    # Each shift is assigned to exactly one nurse.
    for d in all_days:
        for s in all_shifts:
            checker = False
            if(sum(nurse_shifts[(n, d, s)] for n in all_nurses) <= 1):
                checker = True
            list_result.append(checker)
                
    # Each shift is assigned to exactly one doctor.      
    for d in all_days:
        for s in all_shifts:
            checker = False
            if(sum(doctor_shifts[(n, d, s)] for n in all_doctors) <= 1):
                checker = True
            list_result.append(checker)
                
    # Each nurse works at most one shift per day.            
    for n in all_nurses:
        for d in all_days:
            checker = False
            if(sum(nurse_shifts[(n, d, s)] for s in all_shifts) <= 1):
                checker = True
            list_result.append(checker)
                
    # Each doctor works at most one shift per day.            
    for n in all_doctors:
        for d in all_days:
            checker = False
            if(sum(doctor_shifts[(n, d, s)] for s in all_shifts) <= 1):
                checker = True
            list_result.append(checker)
    
    #Each nurse who works on the last shift 
    #will not work on the first shift of the following day    
    for n in all_nurses:
        for d in all_days:
            checker = False
            if(sum([nurse_shifts[(n, d, num_shifts - 1)], nurse_shifts[(n, (d + 1) % 7, 0)]]) <= 1):
                checker = True
            list_result.append(checker)
      
    #Each doctor who works on the last shift 
    #will not work on the first shift of the following day
    for n in all_doctors:
        for d in all_days:
            checker = False
            if(sum([doctor_shifts[(n, d, num_shifts - 1)], doctor_shifts[(n, (d + 1) % 7, 0)]]) <= 1):
                checker = True
            list_result.append(checker)

    #No specialist doctor will work on the last shift
    for n in range(num_doctors - num_doctor_special, num_doctors):
        for d in all_days:
            checker = False
            if(sum([doctor_shifts[(n, d, num_shifts - 1)]]) == 0):
                checker = True
            list_result.append(checker)
    
    #No nurse got overworked (uneven scheduling)          
    min_shifts_per_nurse = (num_shifts * num_days) // num_nurses
    if num_shifts * num_days % num_nurses == 0:
        max_shifts_per_nurse = min_shifts_per_nurse
    else:
        max_shifts_per_nurse = min_shifts_per_nurse + 1
    for n in all_nurses:
        num_shifts_worked = 0
        checker = False
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked += nurse_shifts[(n, d, s)]
        if num_shifts_worked < max_shifts_per_nurse:
            checker = True
        list_result.append(checker)
        
    #No doctor got overworked (uneven scheduling)
    min_shifts_per_doctor = (num_shifts * num_days) // num_doctors
    if num_shifts * num_days % num_doctors == 0:
        max_shifts_per_doctor = min_shifts_per_doctor
    else:
        max_shifts_per_doctor = min_shifts_per_doctor + 1
    for n in all_doctors:
        num_shifts_worked = 0
        checker = False
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked += doctor_shifts[(n, d, s)]
        if num_shifts_worked < max_shifts_per_doctor:
            checker = True
        list_result.append(checker)
    
    if all(list_result):
        print("Constraint satisfied")
    else:
        print("Constraint not satisfied")

In [15]:
ConstraintChecker()

Constraint satisfied
